In [1]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/mdb_eval")
import src.data.aggregators as agg
import src.data.make_data as md
import src.data.selectors as sl
import src.helpers.data as hd
import src.helpers.io as io

fs = s3fs.S3FileSystem(profile="3di")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

## load data

In [2]:
dfa = hd.read_analysis_data("XX0")
hd.inspect(dfa)

shape: (18,817, 32), users: 677


,user_id,ym,ymn,month,txns_count,txns_volume,month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows,user_reg_ym,t,tt,month_spend,age,is_female,region,is_urban,region_code,has_savings_account,has_current_account,generation,generation_code,prop_credit,discret_spend,accounts_active,accounts_total
0,447820,2017-01,201701,1,139.0,8498.040039,2245.067383,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2018-07,0,-18,1562.160034,35.0,0.0,london,1.0,4,True,True,Millennials,3,0.0,905.159973,1,5
1,447820,2017-02,201702,2,119.0,6800.819824,2245.067383,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2018-07,0,-17,2042.869995,35.0,0.0,london,1.0,4,True,True,Millennials,3,0.0,663.750000,1,5


In [3]:
dft = io.read_parquet("s3://3di-data-mdb/clean/mdb_111.parquet")
hd.inspect(dft)

shape: (593,133, 35), users: 270


,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym
0,2012-01-03,111,-114.0,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397323,False,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01
1,2012-01-03,111,8.5,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397324,True,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01


# dev

In [19]:
dfa.notna().all().all()

True

In [14]:
k = agg.discretionary_spend(dft)
k

dspend  dspend_count  dspend_mean
user_id ym                                            
111     2012-01   51.639999             4    12.910000
        2012-02    0.000000             0          NaN
        2012-03   94.650002             2    47.325001
        2012-04  256.649994             6    42.775002
        2012-05  150.990005             5    30.198000
...                     ...           ...          ...
589111  2020-03  952.900024            55    17.325455
        2020-04  438.730011            28    15.668929
        2020-05  199.500000            23     8.673913
        2020-06  216.520004            30     7.217333
        2020-07  209.660004            32     6.551875

[7303 rows x 3 columns]

In [22]:
required_periods = set(range(-6, 6))
required_periods

k.groupby("user_id").filter(lambda g: required_periods.issubset(g.unique())).tail(30)

user_id  ym     
564111   2018-03   -22
         2018-04   -21
         2018-05   -20
         2018-06   -19
         2018-07   -18
         2018-08   -17
         2018-09   -16
         2018-10   -15
         2018-11   -14
         2018-12   -13
         2019-01   -12
         2019-02   -11
         2019-03   -10
         2019-04    -9
         2019-05    -8
         2019-06    -7
         2019-07    -6
         2019-08    -5
         2019-09    -4
         2019-10    -3
         2019-11    -2
         2019-12    -1
         2020-01     0
         2020-02     1
         2020-03     2
         2020-04     3
         2020-05     4
         2020-06     5
         2020-07     6
         2020-08     7
Name: tt, dtype: int64

In [31]:
dfa.user_reg_ym.ge("2017-04").mean()

0.9973956429563564

In [37]:
dfa.groupby("t").tt.describe().T

t,0,1
count,8574.000000,6578.000000
mean,-10.710870,6.508361
std,8.352663,4.985124
min,-35.000000,0.000000
25%,-15.000000,3.000000
50%,-8.000000,6.000000
75%,-4.000000,10.000000
max,-1.000000,24.000000


In [38]:
pd.crosstab(dfa.tt, dfa.t)

t,0,1
tt,,
-35,47,0
-34,53,0
-33,58,0
-32,65,0
-31,68,0
-30,74,0
-29,78,0
-28,86,0
-27,88,0


In [40]:
k = dfa.query("(t == 1) & (tt == 0)")[["user_id", "ym", "t", "tt", "user_reg_ym"]]
k

,user_id,ym,t,tt,user_reg_ym
6,448040,2018-07,1,0,2018-07
35,448070,2018-07,1,0,2018-07
65,448740,2018-07,1,0,2018-07
99,448910,2018-07,1,0,2018-07
119,449550,2018-07,1,0,2018-07
...,...,...,...,...,...
15009,571930,2020-02,1,0,2020-02
15050,572280,2020-02,1,0,2020-02
15091,573040,2020-02,1,0,2020-02
15105,573210,2020-02,1,0,2020-02
